<a href="https://github.com/pytosha/new-python-repo/blob/main/HomeTask">Похожий пример</a><a target="_blank" href="https://colab.research.google.com/github/pytosha/new-python-repo/blob/main/HomeTask/HH-api.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Задание
* Cкачайте все вакансии СБЕРа в Москве api hh.ru (https://github.com/hhru/api у апи есть ограничение в 2000, если будет мешать, попробуйте его обойти)
* Извлеките все полные описания этих вакансий по запросу с https://api.hh.ru/vacancies/43326020 ,где 43326020 id вакансии
* Создайте DataFrame в котором будут даты публикации вакансий, названия, описания,только добавьте поле ключевые навыки


In [4]:
# Загрузка библиотек
import requests
from tqdm.auto import tqdm

In [5]:
# Идентификатор компании
sber = '3529'
# Номер страницы
page = 1
# Количество вакансий на страницу
num_per_page = 100
# Идентификатор области
moscow = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'

In [6]:
# Отправляем GET запрос в api.hh.ru
res = requests.get(url)
# Преобразовываем ответ api из JSON в словарь dict()
vacancies = res.json()
# Получаем количество страниц по результатам запроса
num_pages = vacancies.get('pages')
num_pages

16

In [7]:
# Вывод ключей словаря
vacancies.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

In [8]:
# Получаем список идентификаторов вакансий из короткого описания вакансий
# с одной страницы
vacancy_ids = [el.get('id') for el in vacancies.get('items')]

In [9]:
# Запускаем цикл для сбора всех идентификаторов вакансий со всех страниц
all_vacancy_ids= []
for i in tqdm(range(num_pages)):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    # заполняем список элементами vacancy_ids с помощью метода extend
    all_vacancy_ids.extend(vacancy_ids)

  0%|          | 0/16 [00:00<?, ?it/s]

In [10]:
# Проверка количества идентификаторов вакансий
len(all_vacancy_ids)

1561

In [12]:
# Получение полной информации о вакансии по идентификатору
# Отправляем запрос с идентификатором вакансии
vac_id = 43326020
url = f'https://api.hh.ru/vacancies/{vac_id}'
res = requests.get(url)
vacancy = res.json()


In [13]:
# Пример полного описания вакансии
vacancy

{'id': '43326020',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Специалист по экономической безопасности',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'experience': {'id': 'between1And3', 'name': 'От 1 года до 3 лет'},
 'schedule': {'id': 'fullDay', 'name': 'Полный день'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': {'id': '3529-3529-prof', 'name': 'Сбер для экспертов'},
 'contacts': None,
 'description': '<p><strong>Обязанности:</strong></p> <p> </p> <ul> <li>Поиск и анализ информации в отношении клиентов/контрагентов Банка для оценки уровня кредитных рисков, рисков деловой репутации, иных рисков сотрудничества с контрагентами, в т.ч. в рамках закупочных процедур, реализации строительных программ, б

In [14]:
# Собираем массив с полными описаниями вакансий по идентификаторам
full_vacancies = []
for vac_id in tqdm(all_vacancy_ids):
    url = f'https://api.hh.ru/vacancies/{vac_id}'
    res = requests.get(url)
    vacancy = res.json()
    full_vacancies.append(vacancy)

  0%|          | 0/1561 [00:00<?, ?it/s]

In [ ]:
# Создаем таблицу DataFrame из всех полученных данных

In [16]:
import pandas as pd

In [17]:
df = pd.json_normalize(full_vacancies)

In [18]:
df.head()

,id,premium,relations,name,insider_interview,response_letter_required,salary,allow_messages,contacts,description,...,salary.from,salary.to,salary.currency,salary.gross,address.metro,test.required,errors,request_id,immediate_redirect_url,immediate_redirect_vacancy_id
0,90500047,False,[],Водитель персональный,NaN,False,NaN,False,NaN,<p>Команда транспортного обеспечения Сбербанка...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90502881,False,[],Руководитель направления,NaN,False,NaN,False,NaN,<p>Расчет и анализ риск-взвешенных активов в с...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90433682,False,[],Специалист по документообороту,NaN,False,NaN,False,NaN,<p>Сбер — это 14 тысяч подразделений по всей с...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90500842,False,[],Руководитель направления,NaN,False,NaN,False,NaN,<p>Команда Отдела методологии ПОД/ФТ в поисках...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90495677,False,[],Стратегический менеджер в центр технологий Мет...,NaN,False,NaN,False,NaN,<p>Центр технологиий Метавёрса СБЕРа - это ком...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.shape

(1561, 88)

In [ ]:
# Сохраняем результаты
# Возможно потребуется установка модуля openpyxl
# !pip install openpyxl

In [23]:
# Сохраняем в csv файл
df.to_csv('vacancies.csv', index=False)
# Сохраняем в эксель-файл
df.to_excel('vacancies.xlsx', index=False)
# Сохраняем в файл в формате pickle
df.to_pickle('vacancies.pkl')
